In [1]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import gmaps
import matplotlib.pyplot as plt
%matplotlib notebook
from app_token import sf_crime_token
from datetime import date, datetime, timedelta
from config import g_key
gmaps.configure(api_key=g_key)

## API Request
Resource: https://dev.socrata.com/foundry/data.sfgov.org/wg3w-h783

In [2]:
# Base URL
base_url = 'https://data.sfgov.org/resource/wg3w-h783.json'

In [3]:
# Query string
token = f'$$app_token={sf_crime_token}'
order = '$order=incident_datetime ASC'
limit = '$limit=1000000'

# Date parameter
lockdown_date = date(2020,3,17)
start_date = lockdown_date-timedelta(days=366)
end_date = lockdown_date+timedelta(days=364)
date_range = f"incident_date between '{start_date}' and '{end_date}'"

In [4]:
# Request URL
request_url = f'{base_url}?{token}&{order}&{limit}&$where={date_range}'

In [5]:
# Store response in DataFrame
data = requests.get(request_url).json()
data_df = pd.DataFrame.from_records(data)

## Data Quality Assessment

In [6]:
# Check uniqueness
row_count = len(data_df.index)
print(f'Total Rows: {row_count}')

row_id_count = data_df['row_id'].nunique()
print(f'Unique Row IDs: {row_id_count}')

incident_id_count = data_df['incident_id'].nunique()
print(f'Unique Incident IDs: {incident_id_count}')

columns = ['incident_id', 'incident_datetime', 'report_datetime', 'incident_number', 'police_district', \
           'analysis_neighborhood', 'latitude', 'longitude']
subset_1_row_count = len(data_df[columns].drop_duplicates().index)
print(f'Subset 1 Rows: {subset_1_row_count}')

columns.append('incident_category')
subset_2_row_count = len(data_df[columns].drop_duplicates().index)
print(f'Subset 2 Rows: {subset_2_row_count}')

Total Rows: 260291
Unique Row IDs: 260291
Unique Incident IDs: 215962
Subset 1 Rows: 215962
Subset 2 Rows: 251582


*Finding: Records are unique and identifiable by 'row_id'. However, each record does not correspond to a unique incident. While each 'incident_id' may have only one value for 'incident_datetime', 'report_datetime', 'incident_number', 'police_district', 'analysis_neighborhood', 'supervisor district', 'latitude', and 'longitude', it may be duplicated on 'incident_category' when the incident involves multiple offenses (e.g., illegal possession of drugs and weapons).*

In [7]:
# Check completeness
data_df.count() / row_count

incident_datetime          1.000000
incident_date              1.000000
incident_time              1.000000
incident_year              1.000000
incident_day_of_week       1.000000
report_datetime            1.000000
row_id                     1.000000
incident_id                1.000000
incident_number            1.000000
report_type_code           1.000000
report_type_description    1.000000
filed_online               0.189668
incident_code              1.000000
incident_category          0.998717
incident_subcategory       0.998717
incident_description       1.000000
resolution                 1.000000
intersection               0.950148
cnn                        0.950148
police_district            1.000000
analysis_neighborhood      0.950148
supervisor_district        0.950148
latitude                   0.950148
longitude                  0.950148
point                      0.950148
cad_number                 0.789440
dtype: float64

*Finding: The dataset is sufficiently comprehensive because it captures key incident details such as date, time, category, and location. Furthermore, the frequency of missing values did not exceed 5% for any significant column. Additional details about the perpetrator and victim would have been interesting.*

In [8]:
# Check timeliness
# Resource: https://www.datasciencemadesimple.com/difference-two-timestamps-seconds-minutes-hours-pandas-python-2/
time_df = data_df[['incident_id', 'report_datetime', 'incident_datetime']].drop_duplicates()
time_df[['report_datetime', 'incident_datetime']] = time_df[['report_datetime', 'incident_datetime']].apply(pd.to_datetime)
time_df['diff_datetime'] = time_df['report_datetime'] - time_df['incident_datetime']
time_df['diff_hours'] = time_df['diff_datetime'] / np.timedelta64(1,'h')

quantiles = np.arange(0.05, 1, 0.05)
for item in quantiles:
    value = time_df['diff_hours'].quantile(item)
    print(f"Quantile {'{:.2f}'.format(item)}: {'{:.2f}'.format(value)} Hours")

Quantile 0.05: 0.00 Hours
Quantile 0.10: 0.00 Hours
Quantile 0.15: 0.00 Hours
Quantile 0.20: 0.05 Hours
Quantile 0.25: 0.12 Hours
Quantile 0.30: 0.23 Hours
Quantile 0.35: 0.50 Hours
Quantile 0.40: 1.10 Hours
Quantile 0.45: 2.03 Hours
Quantile 0.50: 3.43 Hours
Quantile 0.55: 6.18 Hours
Quantile 0.60: 11.13 Hours
Quantile 0.65: 15.60 Hours
Quantile 0.70: 21.03 Hours
Quantile 0.75: 30.72 Hours
Quantile 0.80: 51.35 Hours
Quantile 0.85: 92.97 Hours
Quantile 0.90: 184.90 Hours
Quantile 0.95: 526.02 Hours


*Finding: Incident reports are filed in a timely manner. The median elapsed time from incident occurrence to report filing is a mere 3 hours and 26 minutes. Q<sub>1</sub> = 7 minutes and Q<sub>3</sub> = 30 hours and 40 minutes. Once filed, incident reports are published via an automated daily update.*

In [9]:
# Check consistency
# Resource: https://towardsdatascience.com/master-the-most-hated-task-in-ds-ml-3b9779276d7c
incident_categories = data_df['incident_category'].value_counts().sort_index()
print(incident_categories)

Arson                                           785
Assault                                       15603
Burglary                                      16111
Case Closure                                   1072
Civil Sidewalks                                 248
Courtesy Report                                 762
Disorderly Conduct                             4432
Drug Offense                                   6062
Drug Violation                                   87
Embezzlement                                    347
Fire Report                                     363
Forgery And Counterfeiting                      926
Fraud                                          7870
Gambling                                          5
Homicide                                         29
Human Trafficking (A), Commercial Sex Acts       23
Human Trafficking, Commercial Sex Acts           13
Larceny Theft                                 76070
Liquor Laws                                      15
Lost Propert

In [10]:
police_districts = data_df['police_district'].value_counts().sort_index()
print(police_districts)

Bayview       23855
Central       35957
Ingleside     20694
Mission       32466
Northern      37023
Out of SF      7563
Park          12890
Richmond      16933
Southern      30257
Taraval       18291
Tenderloin    24362
Name: police_district, dtype: int64


In [11]:
neighborhoods = data_df['analysis_neighborhood'].value_counts().sort_index()
print(neighborhoods)

Bayview Hunters Point             16488
Bernal Heights                     4831
Castro/Upper Market                7684
Chinatown                          4485
Excelsior                          4456
Financial District/South Beach    19211
Glen Park                          1463
Golden Gate Park                   2533
Haight Ashbury                     4264
Hayes Valley                       7297
Inner Richmond                     3385
Inner Sunset                       3300
Japantown                          2331
Lakeshore                          2911
Lincoln Park                        404
Lone Mountain/USF                  3214
Marina                             7393
McLaren Park                        317
Mission                           25928
Mission Bay                        4495
Nob Hill                           7515
Noe Valley                         2839
North Beach                        6710
Oceanview/Merced/Ingleside         2539
Outer Mission                      3750


*Finding: In 'incident_category', multiple inconsistencies exists: the word "Offense" is sometimes spelled "Offence", a "?" is appended to "Motor Vehicle Theft", and "(A)" is inserted into "Human Trafficking, Commercial Sex Acts." Furthermore, the field requires grouping of several categories: "Drug Violation" into "Drug Offense", "Other" and "Other Offenses" into "Other Miscellaneous", "Suspicious" into "Suspicious Occ", and "Weapons Carrying Etc" into "Weapons Offense". In 'analysis_neighborhood', an inconsistency exists across "null" and blank values; they are two representations of the same value.*

## Data Cleansing

In [12]:
# Fill missing values
data_df.fillna({'analysis_neighborhood':'Not Disclosed', 'incident_category':'Not Disclosed'}, inplace=True)
data_df['analysis_neighborhood'].replace('null','Not Disclosed', inplace=True, regex=True)

In [13]:
# Fix typos
data_df['incident_category'].replace(['Offence','\?',' \(A\)'],['Offense','',''], inplace=True, regex=True)

In [14]:
# Group similar categories
data_df['incident_category'].replace(['Drug Violation','Other','Other Offenses','Suspicious','Weapons Carrying Etc'], \
                                     ['Drug Offense','Other Miscellaneous','Other Miscellaneous','Suspicious Occ', 'Weapons Offense'], inplace=True)

In [15]:
# Split by pre and post lockdown
pre_df = data_df.loc[data_df['incident_date'] < lockdown_date.strftime('%Y-%m-%d')]
post_df = data_df.loc[data_df['incident_date'] >= lockdown_date.strftime('%Y-%m-%d')]

## Data Analysis

**Category Analysis**

In [ ]:
# Group total incidents by category
pre_cat_ser = pre_df.groupby('incident_category')['incident_id'].nunique().rename('pre_incidents')
post_cat_ser = post_df.groupby('incident_category')['incident_id'].nunique().rename('post_incidents')

In [ ]:
# Concatenate pre and post incidents, calculate percent change
cat_df = pd.concat([pre_cat_ser, post_cat_ser], sort=True, axis=1).fillna(0)
cat_df['post_incidents'] = cat_df['post_incidents'].astype(int)
cat_df.drop(labels='Not Disclosed', axis=0, inplace=True)
cat_df['percent_change'] = (cat_df['post_incidents'] - cat_df['pre_incidents']) / cat_df['pre_incidents'] * 100
cat_df.sort_values(by=['post_incidents'], ascending=True, inplace=True)
cat_df.index = cat_df.index + ' (' + cat_df['post_incidents'].astype(str) + ')'

In [ ]:
# Plot
plt.figure(figsize=(10,10))
plt.barh(cat_df.index, cat_df['percent_change'], height=0.5, color='royalblue', align='center')
plt.title(f'Category Incidents Post-Lockdown')
plt.xlabel('Percent Change YOY')
plt.ylabel('Category (Incidents)')
plt.xlim(-100, max(100, cat_df['percent_change'].max()))
plt.ylim(-0.5, len(cat_df.index)-0.5)
plt.tick_params(axis='y',left=False)
plt.grid(axis='x', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.savefig('img/incidents_category.png')

**Narrow Scope to Crimes of Stolen Property Only**

In [ ]:
stolen_types = ['Burglary', 'Larceny Theft', 'Lost Property', 'Motor Vehicle Theft', 'Robbery', 'Stolen Property']
#stolen_types = ['Burglary']
stolen_pre_df = pre_df.loc[data_df['incident_category'].isin(stolen_types)]
stolen_post_df = post_df.loc[data_df['incident_category'].isin(stolen_types)]

**Map Analysis**<br>
Resource: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

In [ ]:
# Map incidents by latitude and longitude
def map_incidents(df):
    latlong_df = df[['incident_id', 'latitude', 'longitude']].copy().drop_duplicates().dropna(how='any')
    locations = latlong_df[['latitude', 'longitude']].astype(float)
    fig_layout_dict = {'width':'1000px', 'height':'1000px', 'border':'1px solid black', 'padding':'5px'}
    fig = gmaps.figure(layout=fig_layout_dict)
    heat_layer = gmaps.heatmap_layer(locations, dissipating=True, max_intensity=5, point_radius=5)
    fig.add_layer(heat_layer)
    return fig

In [ ]:
# Map pre-lockdown incidents
map_incidents(stolen_pre_df)

In [ ]:
# Map post-lockdown incidents
map_incidents(stolen_post_df)

**Neighborhood Analysis**

In [ ]:
# Group incidents by neighborhood
def chart_neighborhood_incidents(df, timing):
    district_ser = df.groupby('analysis_neighborhood')['incident_id'].nunique()
    district_ser.sort_values(ascending=True, inplace=True)
    plt.figure(figsize=(10,10))
    plt.barh(district_ser.index, district_ser, height=0.5, color='royalblue', align='center')
    plt.title(f'Neighborhood Incidents {timing}-Lockdown')
    plt.xlabel('Incidents')
    plt.ylabel('Neighborhood')
    plt.ylim(-0.5, len(district_ser)-0.5)
    plt.tick_params(axis='y',left=False)
    plt.tight_layout()
    plt.savefig(f'img/incidents_neighborhood_{timing.lower()}.png')

In [ ]:
# Chart pre-lockdown incidents
chart_neighborhood_incidents(stolen_pre_df, 'Pre')

In [ ]:
# Chart post-lockdown incidents
chart_neighborhood_incidents(stolen_post_df, 'Post')

**Time Analysis**

In [ ]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [ ]:
# Group total incidents by day and hour
def chart_time_incidents(start_date, end_date, df, timing):
    
    # Count occurrence of each day of week within date range
    # Resource: https://numpy.org/doc/stable/reference/generated/numpy.busday_count.html
    days_dict = {}
    def day_counter(start_date, end_date, day):
        day_count = np.busday_count(start_date, end_date+timedelta(days=1), weekmask=day)
        return day_count
    for day in days:
        days_dict[day] = day_counter(start_date, end_date, day)
    
    # Group total incidents by day and hour
    # Resource: https://www.javaer101.com/en/article/17171715.html
    time_df = df[['incident_day_of_week', 'incident_time', 'incident_id']].copy().drop_duplicates()
    day_cats = pd.api.types.CategoricalDtype(categories=days, ordered=True)
    time_df['day_abbr'] = time_df['incident_day_of_week'].str[:3].astype(day_cats)
    time_df['incident_hour'] = time_df['incident_time'].str[:2].astype(int)
    time_ser = time_df.groupby(['day_abbr', 'incident_hour'])['incident_id'].nunique().astype(float)
    
    # Convert total incidents to average incidents by weekday and hour
    def day_count_divisor(day):
        time_ser[day] = time_ser.loc[day].divide(days_dict[day], level='day_abbr')
    for day in days:
        day_count_divisor(day)

    # Plot
    time_ser.unstack(level=0).plot(
        kind='line',
        figsize=(12,8),
        title=f'Average Hourly Incidents {timing}-Lockdown')
    plt.xlabel('Hour')
    plt.ylabel('Average Incidents')
    plt.ylim(0, time_ser.max()*1.2)
    plt.xticks(np.arange(0,24,2))
    plt.legend(title='Day of Week')
    plt.tight_layout()
    plt.savefig(f'img/incidents_hourly_{timing.lower()}.png')

In [ ]:
chart_time_incidents(start_date, lockdown_date-timedelta(days=1), stolen_pre_df, 'Pre')

In [ ]:
chart_time_incidents(lockdown_date, end_date, stolen_post_df, 'Post')